<a href="https://colab.research.google.com/github/jcs-lambda/DS-Unit-2-Kaggle-Challenge/blob/master/module3/LS_DS_223_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 2, Module 3*

---

# Cross-Validation


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.


You won't be able to just copy from the lesson notebook to this assignment.

- Because the lesson was ***regression***, but the assignment is ***classification.***
- Because the lesson used [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html), which doesn't work as-is for _multi-class_ classification.

So you will have to adapt the example, which is good real-world practice.

1. Use a model for classification, such as [RandomForestClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
2. Use hyperparameters that match the classifier, such as `randomforestclassifier__ ...`
3. Use a metric for classification, such as [`scoring='accuracy'`](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values)
4. If you’re doing a multi-class classification problem — such as whether a waterpump is functional, functional needs repair, or nonfunctional — then use a categorical encoding that works for multi-class classification, such as [OrdinalEncoder](https://contrib.scikit-learn.org/categorical-encoding/ordinal.html) (not [TargetEncoder](https://contrib.scikit-learn.org/categorical-encoding/targetencoder.html))



## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
import pandas as pd

# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [0]:
import pandas as pd
import numpy as np
import pandas_profiling
import category_encoders as ce
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import graphviz
import itertools
import re
import random

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import Ridge, RidgeCV, LinearRegression, LogisticRegression, LogisticRegressionCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
from sklearn.ensemble import RandomForestClassifier

from google.colab import files
from google.colab.data_table import DataTable

In [0]:
pd.options.display.max_rows = 999
pd.options.display.max_columns = 100

In [0]:
random_state = 13

In [0]:
def wrangle(dataframe):
  df = dataframe.copy()

  # date_recorded to pandas datetime objects
  df['date_recorded'] = pd.to_datetime(df['date_recorded'], infer_datetime_format=True, errors='coerce')

  # source - combine 'unknown' with 'other'
  df['source'] = df['source'].str.replace('unknown', 'other')

  return df

In [0]:
df_train = wrangle(train)
df_test = wrangle(test)

In [0]:
target = 'status_group'

In [14]:
train.columns

Index(['id', 'amount_tsh', 'date_recorded', 'funder', 'gps_height',
       'installer', 'longitude', 'latitude', 'wpt_name', 'num_private',
       'basin', 'subvillage', 'region', 'region_code', 'district_code', 'lga',
       'ward', 'population', 'public_meeting', 'recorded_by',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type', 'extraction_type_group', 'extraction_type_class',
       'management', 'management_group', 'payment', 'payment_type',
       'water_quality', 'quality_group', 'quantity', 'quantity_group',
       'source', 'source_type', 'source_class', 'waterpoint_type',
       'waterpoint_type_group', 'status_group'],
      dtype='object')

In [0]:
features = train.columns.drop([target, 'id', 'date_recorded', 'recorded_by', 'quality_group', 'quantity_group', 'waterpoint_type_group', 'source_type', 'source_class', 'payment_type', 'management_group', 'extraction_type_group', 'extraction_type_class'])

In [0]:
X_train = df_train[features]
X_test = df_test[features]

y_train = df_train[target]

In [17]:
y_pred = [y_train.mode()[0]] * len(y_train)
baseline_majority_accuracy = accuracy_score(y_train, y_pred)
print(f'Baseline majority accuracy: {baseline_majority_accuracy * 100:.2f}%')

Baseline majority accuracy: 54.31%


In [0]:
cardinality = X_train.select_dtypes(exclude='number').nunique()
low_cardinality = cardinality[cardinality < 51].index.to_list()
high_cardinality = cardinality[cardinality > 50].index.to_list()

In [0]:
pipeline = make_pipeline(
    ce.OneHotEncoder(cols=low_cardinality, use_cat_names=True),
    ce.OrdinalEncoder(cols=high_cardinality),
    SimpleImputer(strategy='median'),
    RandomForestClassifier(n_estimators=100, random_state=random_state, n_jobs=-1, )
)

In [22]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('onehotencoder',
                 OneHotEncoder(cols=['basin', 'region', 'public_meeting',
                                     'scheme_management', 'permit',
                                     'extraction_type', 'management', 'payment',
                                     'water_quality', 'quantity', 'source',
                                     'waterpoint_type'],
                               drop_invariant=False, handle_missing='value',
                               handle_unknown='value', return_df=True,
                               use_cat_names=True, verbose=0)),
                ('ordinalenco...
                ('randomforestclassifier',
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                             

In [23]:
y_pred_1 = pipeline.predict(X_test)
len(y_pred_1)

14358

In [0]:
from sklearn.model_selection import RandomizedSearchCV

In [0]:
param_distributions = {
    'simpleimputer__strategy' : ['mean', 'median'],
    'randomforestclassifier__n_estimators' : [x for x in range(10, 101, 10)],
    'randomforestclassifier__class_weight' : ['balanced', 'balanced_subsample'],
    'randomforestclassifier__bootstrap' : [True, False],
    'randomforestclassifier__max_depth' : [None, 10, 20, 40, 80, 100],
    'randomforestclassifier__min_samples_split' : [2, 4, 8, 16, 32],
    'randomforestclassifier__min_samples_leaf' : [1, 10, 20]
}

In [0]:
search = RandomizedSearchCV(pipeline, param_distributions=param_distributions, n_iter=100, cv=5, n_jobs=-1, verbose=10)

In [33]:
search.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   17.7s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done  57 tasks      | elapsed:  7.6min
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:  9.3min
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed: 10.8min
[Parallel(n_jobs=-1)]: Done  94 tasks      | elapsed: 12.7min
[Parallel(n_jobs=-1)]: Done 109 tasks      | elapsed: 13.8min
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed: 15.4min
[Parallel(n_jobs=-1)]: Done 141 tasks      | elapsed: 17

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('onehotencoder',
                                              OneHotEncoder(cols=['basin',
                                                                  'region',
                                                                  'public_meeting',
                                                                  'scheme_management',
                                                                  'permit',
                                                                  'extraction_type',
                                                                  'management',
                                                                  'payment',
                                                                  'water_quality',
                                                                  'quantity',
                           

In [0]:
best_pipeline = search.best_estimator_

In [35]:
y_pred_2 = best_pipeline.predict(X_test)
len(y_pred_2)

14358

In [36]:
file_name = 'submission.csv'

submission = sample_submission.copy()
submission['status_group'] = y_pred_2

submission.shape

(14358, 2)

In [37]:
submission.to_csv(file_name, index=False, header=True)
!head "$file_name"

id,status_group
50785,functional
51630,functional
17168,functional
45559,non functional
49871,functional
52449,functional
24806,functional
28965,non functional
36301,non functional


In [0]:
files.download(file_name)